# Libraries

In [8]:
# Libraries
import requests
import json
import pandas as pd
import folium
from shapely.geometry import shape
from shapely.ops import transform
import geopandas as gpd
from pyproj import CRS, Transformer

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

/workspaces/project_arm_airbnb/data_maps


## API request

### Touristenattraktionen

In [15]:
pip install overpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import overpy
import folium

# Step 1: Initialize Overpass API
api = overpy.Overpass()

# Step 2: Define the Overpass query for tourism attractions in Geneva
query = """
[out:json][timeout:25];
area[name="Genf"]->.searchArea;
(
  node["tourism"](area.searchArea);
  way["tourism"](area.searchArea);
  relation["tourism"](area.searchArea);
);
out center;
"""

# Step 3: Execute the query
result = api.query(query)

# Step 4: Create a base map centered on Geneva
geneva_map = folium.Map(location=[46.2044, 6.1432], zoom_start=13)  # Geneva's coordinates

# Step 5: Add nodes (points of interest) to the map
for node in result.nodes:
    name = node.tags.get("name", "Unknown")
    folium.Marker(
        location=[node.lat, node.lon],
        popup=f"{name} (Node)",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(geneva_map)

# Step 6: Add ways (areas/paths of interest) to the map
for way in result.ways:
    name = way.tags.get("name", "Unknown")
    if way.nodes:
        coordinates = [(node.lat, node.lon) for node in way.nodes]
        folium.PolyLine(
            locations=coordinates,
            color="red",
            weight=2,
            popup=f"{name} (Way)"
        ).add_to(geneva_map)

# Step 7: Add relations (complex areas of interest) to the map
for relation in result.relations:
    name = relation.tags.get("name", "Unknown")
    if "center" in relation:
        lat, lon = relation.center_lat, relation.center_lon
        folium.Marker(
            location=[lat, lon],
            popup=f"{name} (Relation)",
            icon=folium.Icon(color="green", icon="star")
        ).add_to(geneva_map)

# Step 8: Save and display the map
geneva_map.save("geneva_tourist_map.html")
print("Map saved as geneva_tourist_map.html")


from IPython.display import IFrame

def plot_map(map_file):
    return IFrame(map_file, width=700, height=500)

plot_map("geneva_tourist_map.html")

Map saved as geneva_tourist_map.html
